In [1]:
import sys
sys.path.append('../../../../../')
import setGPU
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import math
import h5py
import qibo
qibo.set_backend("tensorflow")
sys.path.append('../')
sys.path.append('../../')
sys.path.append('/eos/home-e/epuljak/private/epuljak/Autoencoders')
import scripts.qkmeans as qkm
import scripts.minimization as m 
import scripts.qkmedians as qkmed
import scripts.oracle as o
import scripts.kmedians as kmed
import utils as u
import plots as p

[Qibo 0.1.6|INFO|2022-08-19 14:15:01]: Using qibojit backend on /GPU:0
[Qibo 0.1.6|INFO|2022-08-19 14:15:08]: Using tensorflow backend on /device:GPU:0


In [2]:
qibo.set_device("/GPU:0")

In [3]:
root_dir='/eos/user/e/epuljak/private/epuljak'

In [ ]:
#p.plot_latent_representations(data_s, cluster_label_s, f'{root_dir}/PhD/TN/QIBO/search_algorithms/plots/latent_dim8', f'latent_feature_pairs_jets_qkmedians_{run}_Durr_DI_AE_SIGNALdata' )

In [ ]:
# data = [q_distances, q_distances_s, c_distances, c_distances_s]
# p.plot_sum_distance_compare(data, name_signal='GtWWna35', normalize=False, id_fig=f'300', save_dir=f'{root_dir}/PhD/TN/QIBO/search_algorithms/plots/latent_dim25')

#### Compare ROCs

In [ ]:
# change classical back to cluster assignments

In [ ]:
run0='29032022_1' #4
run1='28032022_1' #5
run2='24032022_1' #6
run3='10022022_1' #8
#run01='25032022_1' #8 second
run4='22022022_1' #16
run5='01032022_1' #24
#run6='04032022_1' #25
#run4='03032022_1' #26
run6='01032022_2' #32
run7='14032022_1' #40

In [ ]:
runs=[run0, run1, run2, run3, run4, run5, run6, run7]
#runs=[run5]*9
#runs=[run5]*2

In [12]:
n_samples_train=[10, 6000]
#n_samples_train=[5, 10, 30, 50, 100, 300, 500, 1000, 10000]
#n_samples_train=[600]*5

In [13]:
#identifiers=['4', '8', '16', '24', '32']
identifiers=['8', '8']
#identifiers=['4 (argmin)', '5 (argmin)', '6 (argmin)', '8 (argmin)', '16 (argmin)', '24 (argmin)', '32 (argmin)', '40 (argmin)']
#identifiers=['5', '10', '30', '50', '100', '300', '500', '1000', '10000']
#identifiers=['24', '24 - Grover']

In [ ]:
clusters=['2', '5', '10', '15']

In [14]:
signal_name='RSGraviton_WW'
#signal_name='AtoHZ_to_ZZZ'

In [15]:
quantum, classic = p.calc_AD_scores(identifiers, n_samples_train, k=2, test_size=10000, signal_name=signal_name, mass='35', br_na='NA', read_test_dir='/eos/user/e/epuljak/private/epuljak/public/AE_data/latent', split=True, n_folds=10)


(2000, 8)
(2000, 8)
(2000, 8)
(2000, 8)
(2000, 8)
(2000, 8)
(2000, 8)
(2000, 8)
(2000, 8)
(2000, 8)
(2000, 8)
(2000, 8)
(2000, 8)
(2000, 8)
(2000, 8)
(2000, 8)
(2000, 8)
(2000, 8)
(2000, 8)
(2000, 8)
Find Cluster Labels ---> 10.91989278793335 seconds ---
Find Cluster Labels ---> 10.853245973587036 seconds ---
Find Cluster Labels ---> 10.847728967666626 seconds ---
Find Cluster Labels ---> 10.795499563217163 seconds ---
Find Cluster Labels ---> 10.77996277809143 seconds ---
Find Cluster Labels ---> 10.900406837463379 seconds ---
Find Cluster Labels ---> 10.65216064453125 seconds ---
Find Cluster Labels ---> 10.876890182495117 seconds ---
Find Cluster Labels ---> 10.758670568466187 seconds ---
Find Cluster Labels ---> 10.812758445739746 seconds ---
Find Cluster Labels ---> 10.684707403182983 seconds ---
Find Cluster Labels ---> 10.670864820480347 seconds ---
Find Cluster Labels ---> 10.711763858795166 seconds ---
Find Cluster Labels ---> 10.666523694992065 seconds ---
Find Cluster Labels

In [ ]:
quantum, classic = p.calc_AD_scores_nclusters(identifiers, n_samples_train, clusters, k=2, test_size=10000, signal_name=signal_name, mass='35', br_na='NA', read_test_dir='/eos/user/e/epuljak/private/epuljak/public/AE_data/latent')


In [ ]:
import importlib
importlib.reload(p)

In [ ]:
q = quantum # old

In [ ]:
p.plot_correlation_pT_AD_score('/eos/user/e/epuljak/private/epuljak/PhD/TN/QIBO/search_algorithms/notebooks/ad_scores/final',\
                              ids=identifiers, n_samples_train=n_samples_train,\
                              signal_name=signal_name, mass='35',br_na='NA',\
                              save_dir=f'{root_dir}/PhD/TN/QIBO/search_algorithms/plots/diJet/train600/GtoWW35NA/final')

In [ ]:
p.plot_ROCs_compare(q, quantum, ids=identifiers,\
                    colors=np.array(['C'+str(j+1)for j in range(5)]),\
                    title=r'$N_{qcd}=10k, N_{sig}=10k$',\
                    save_dir = f'{root_dir}/PhD/TN/QIBO/search_algorithms/plots/diJet/train600/GtoWW35NA/final')

In [ ]:
# AUC Q/C ratios
n_folds=10
read_dir='/eos/user/e/epuljak/private/epuljak/PhD/TN/QIBO/search_algorithms/notebooks/ad_scores/final'
mass='35'
br_na='NA'

q_loss_qcd=[]; q_loss_sig=[]; c_loss_qcd=[]; c_loss_sig=[]
for i in range(len(identifiers)):
    if br_na: 
        with h5py.File(f'{read_dir}/{identifiers[i]}/Latent_{identifiers[i]}_trainsize_{n_samples_train[i]}_{signal_name}{mass}{br_na}_nfolds{n_folds}.h5', 'r') as file:
            q_loss_qcd.append(file['quantum_loss_qcd'][:])
            q_loss_sig.append(file['quantum_loss_sig'][:])
            c_loss_qcd.append(file['classic_loss_qcd'][:])
            c_loss_sig.append(file['classic_loss_sig'][:])

In [ ]:
import plotting as pl

In [ ]:
import importlib
importlib.reload(pl)

In [ ]:
pl.create_table_for_fixed_TPR(q_loss_qcd, q_loss_sig, c_loss_qcd, c_loss_sig, identifiers, 10)

In [ ]:
pl.plot_ROC_kfold_mean(q_loss_qcd, q_loss_sig, c_loss_qcd, c_loss_sig, identifiers, 10,\
                colors=np.array(['C'+str(j+1)for j in range(6)]),\
                title=r'$N_{qcd}=10k, N_{sig}=10k$')

In [ ]:
p.plot_ratio_QC_auc_kfold(q_loss_qcd, q_loss_sig, c_loss_qcd, c_loss_sig, identifiers, n_folds, xlabel='Latent dimensions', title=None, legend_loc='best', save_dir=f'{root_dir}/PhD/TN/QIBO/search_algorithms/plots/diJet/train600/AtoHZtoZZZ35/final')

In [ ]:
p.plot_ROC_kfold(q_loss_qcd, q_loss_sig, c_loss_qcd, c_loss_sig, identifiers, 10, pic_id='errs_nfolds10_quantum_fprtpr_noprop',\
                colors=np.array(['C'+str(j+1)for j in range(6)]),\
                title=r'$N_{qcd}=2k, N_{sig}=2k$', save_dir = f'{root_dir}/PhD/TN/QIBO/search_algorithms/plots/diJet/train600/GtoWW35NA/final')

In [ ]:
#background, signal = p.calculate_ROCs(runs, n_samples_train, identifiers, qcd_test_size=500, n_samples_signal=500, br_na='NA', signal_name=signal_name, mass='3.5')
background, signal = p.calculate_ROCs(runs, n_samples_train, identifiers, qcd_test_size=500, n_samples_signal=500, signal_name=signal_name, mass='3.5', br_na='NA')

In [ ]:
#load_filename=f'{root_dir}/PhD/TN/QIBO/search_algorithms/notebooks/results_qmedians/corrected_cuts/centroids/centroids_{runs[0]}_Durr_DI_AE_{str(n_samples_train[0])}_correctedcuts.npy'
background, signal = p.calculate_ROCs(runs, n_samples_train, identifiers, qcd_test_size=500, n_samples_signal=500, br_na='BR', signal_name=signal_name, mass='1.5', around_peak='around15')
#background, signal = p.calculate_ROCs(runs, n_samples_train, identifiers, qcd_test_size=500, n_samples_signal=500, signal_name=signal_name, mass='3.5')


In [ ]:
background, signal = p.calculate_ROCs_kfold(runs, n_samples_train, identifiers, signal_name=signal_name, mass='3.5')

In [ ]:
background_Q, signal_Q, background_C, signal_C = p.calculate_ROCs_randomVStrained(runs,\
                                                                                  n_samples_train,\
                                                                                  identifiers,\
                                                                                  qcd_test_size=500, n_samples_signal=500,\
                                                                                  br_na='NA',\
                                                                                  signal_name=signal_name,\
                                                                                  mass='3.5')


In [ ]:
p.plot_rocs_QKmedians_compare(background, signal,\
                              legend_loc='upper left',\
                              ix='final_around15_log',\
                              n=len(runs),\
                              colors=np.array(['C'+str(j+1)for j in range(len(runs))]),\
                              ids=identifiers,\
                              title=r'$N_{qcd}=0.5k, N_{sig}=0.5k$',\
                              save_dir=f'{root_dir}/PhD/TN/QIBO/search_algorithms/plots/train500/corrected_cuts/GtoWW1.5BR')

In [ ]:
data = [background[0], signal[0], background[1], signal[1]]
p.plot_sum_distance_compare(data, name_signal='GtWWna3.5', normalize=False, id_fig=f'300', save_dir='/eos/user/e/epuljak/private/epuljak/PhD/TN/QIBO/search_algorithms/plots/latent_dim40/corrected_cuts/')

In [ ]:
p.plot_rocs_QKmedians_compare(background_Q, signal_Q,\
                              legend_loc='upper left',\
                              ix='RvsTrained10k_Q',\
                              n=len(runs),\
                              colors=np.array(['C'+str(j+1)for j in range(len(runs))]),\
                              ids=identifiers,\
                              title=r'$Q-medians$',\
                              save_dir=f'{root_dir}/PhD/TN/QIBO/search_algorithms/plots/latent_dim24/corrected_cuts/GtoWW3.5NA')

In [ ]:
p.plot_rocs_QKmedians_compare(background_C, signal_C,\
                              legend_loc='upper left',\
                              ix='RvsTrained10k_C',\
                              n=len(runs),\
                              colors=np.array(['C'+str(j+1)for j in range(len(runs))]),\
                              ids=identifiers,\
                              title=r'$C-kmedians$',\
                              save_dir=f'{root_dir}/PhD/TN/QIBO/search_algorithms/plots/latent_dim24/corrected_cuts/GtoWW3.5NA')

In [ ]:
p.plot_auc_fpr(quantum, classic,\
               ids=identifiers,\
               tpr_window=[0.4, 0.5],\
               xlabel='Latent Dimensions',\
               n=len(identifiers),\
               ix='diJet',\
               save_dir=f'{root_dir}/PhD/TN/QIBO/search_algorithms/plots/diJet/train600/GtoWW35NA/around35')

In [ ]:
p.plot_auc_kfold(background, signal, ids=identifiers, n=len(runs), n_folds=4, xlabel='Latent dimensions', legend_loc='best',\
              save_dir=f'{root_dir}/PhD/TN/QIBO/search_algorithms/plots/train500/corrected_cuts/AtoHZtoZZZ3.5')

## Plot LOSS vs EPOCHS

### Quantum

In [ ]:
run = '01032022_1'
n_samples_train=5

In [ ]:
loss_q = np.load(f'{root_dir}/PhD/TN/QIBO/search_algorithms/notebooks/results_qmedians/corrected_cuts/LOSS_{run}_argmin_DI_AE_{str(n_samples_train)}_correctedcuts.npy')

In [ ]:
epochs = np.array((range(loss_q.shape[0])))+1

In [ ]:
save_dir = f'{root_dir}/PhD/TN/QIBO/search_algorithms/plots/diJet/train600/GtoWW35NA/final'
ix = 'minClassic_2_clusterassign'

In [ ]:
loss_q = np.load(f'/eos/user/e/epuljak/private/epuljak/PhD/TN/QIBO/search_algorithms/notebooks/results_qmedians/corrected_cuts/diJet/loss/LOSS_lat32_600.npy')

In [ ]:
q_loss_qcd = np.array(q_loss_qcd)

In [ ]:
fig = plt.figure(figsize=(10,10))
fig.subplots_adjust(hspace=0.4, wspace=0.4)
#for i, ids in enumerate(identifiers):
for i in range(5):
    #loss_q = np.load(f'{root_dir}/PhD/TN/QIBO/search_algorithms/notebooks/results_qmedians/corrected_cuts/diJet/loss/final/LOSS_lat{identifiers[i]}_{n_samples_train[i]}.npy')
    loss_q = q_loss_qcd[i,:]
    #epochs = np.array((range(loss_q.shape[0])))+1
    # ax = fig.add_subplot(2,3, i+1)
    # ax.plot(epochs[:], loss_q[:])
    # ax.title.set_text(f'Run: lat{ids}; #samples:{str(n_samples_train[i])}')
    # ax.title.set_size(10)
plt.xlabel('Epochs')
plt.ylabel('Loss')
#plt.savefig(f'{save_dir}/Quantum_training_loss_latdim_final.pdf', bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=(8,8))
plt.hist(q_loss_qcd[0][0], bins=50, density=True, histtype = 'step', label='qcd')
plt.hist(q_loss_sig[0][0], bins=50, density=True, histtype = 'step', label='bsm')

plt.legend()
#plt.yscale('log')
plt.show()

In [ ]:
q_loss_qcd.shape

In [ ]:
fig = plt.figure()
epochs = np.array((range(loss_q.shape[0])))+1
plt.plot(epochs[5:], loss_q[5:])
plt.xlabel('Epochs')
plt.ylabel('Loss')
#plt.savefig(f'{save_dir}/Quantum_training_loss_trainsize.pdf', bbox_inches='tight')
plt.show()

### Classic

In [ ]:
run = '24032022_1'
n_samples_train=500
signal_name='RSGraviton_WW_NA'
mass='3.5'
br_na='NA'

In [ ]:
#_,_,centroids_c,loss_c = u.load_data_and_centroids_c(run, n_samples_train=n_samples_train, qcd_test_size=500, n_samples_test=500, signal_name=signal_name, mass=mass, br_na=br_na)
loss_c = np.load(f'{root_dir}/PhD/TN/QIBO/search_algorithms/notebooks/results_qmedians/corrected_cuts/LOSS_{run}_{str(n_samples_train)}_classic.npy')

In [ ]:
save_dir = f'{root_dir}/PhD/TN/QIBO/search_algorithms/notebooks/results_qmedians/corrected_cuts'
#np.save(f'{save_dir}/LOSS_{run}_Durr_DI_AE_{str(n_samples_train)}_Ckmeans_sameRandomStart.npy', loss_c)
#np.save(f'{save_dir}/centroids/centroids_{run}_Durr_DI_AE_{str(n_samples_train)}_classicKmeans_sameRandomStart.npy', centroids_c)

In [ ]:
epochs = np.array((range(len(loss_c))))+1

In [ ]:
fig = plt.figure()
fig.subplots_adjust(hspace=0.4, wspace=0.4)
for i in range(1, 10):
    loss_q = np.load(f'{root_dir}/PhD/TN/QIBO/search_algorithms/notebooks/results_qmedians/corrected_cuts/LOSS_{runs[i-1]}_{str(n_samples_train[i-1])}_classic_centroids_conv.npy')
    epochs = np.array((range(loss_q.shape[0])))+1
    ax = fig.add_subplot(3, 3, i)
    ax.plot(epochs[:], loss_q[:])
    ax.title.set_text(f'Run:{runs[i-1]}; #samples:{str(n_samples_train[i-1])}')
    ax.title.set_size(10)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.savefig(f'{save_dir}/Classic_training_loss_trainsize.pdf', bbox_inches='tight')
plt.show()

### Compare centroids

In [ ]:
centroids_q = np.load(f'{root_dir}/PhD/TN/QIBO/search_algorithms/notebooks/results_qmedians/corrected_cuts/centroids/centroids_{run}_Durr_DI_AE_{str(n_samples_train)}_correctedcuts.npy')
centroids_c = np.load(f'{root_dir}/PhD/TN/QIBO/search_algorithms/notebooks/results_qmedians/corrected_cuts/centroids/centroids_{run}_{str(n_samples_train)}_classic.npy')

In [ ]:
p.plot_centroids_compare(centroids_q[:,:10], centroids_c[:,:10], fig_dir=f'{root_dir}/PhD/TN/QIBO/search_algorithms/plots/latent_dim40/corrected_cuts', sample_id='lat40')

In [ ]:
#fig = plt.figure()
#fig.subplots_adjust(hspace=0.4, wspace=0.4)
for i in range(0,20):
    centroids = np.load(f'{root_dir}/PhD/TN/QIBO/search_algorithms/notebooks/results_qmedians/corrected_cuts/diJet/centroids/final/check_loss/centroids_lat8_6000_i{i+1}.npy')
    p.plot_centroids(centroids, f'{root_dir}/PhD/TN/QIBO/search_algorithms/notebooks/results_qmedians/corrected_cuts/diJet/centroids/final/check_loss', sample_id=f'lat8_6000_i{i+1}')

In [ ]:
centroids = np.load(f'{root_dir}/PhD/TN/QIBO/search_algorithms/notebooks/results_qmedians/corrected_cuts/diJet/centroids/final/centroids_lat8_600_k3.npy')
print(centroids.shape)
p.plot_centroids(centroids, f'{root_dir}/PhD/TN/QIBO/search_algorithms/notebooks/results_qmedians/corrected_cuts/diJet/centroids/final', sample_id='k3', clusters=3)

In [ ]:
class_labels = np.load(f'{root_dir}/PhD/TN/QIBO/search_algorithms/notebooks/results_qmedians/corrected_cuts/diJet/cluster_labels/final/cluster_label_lat8_600_k3.npy')


## Check code

In [ ]:
import importlib
importlib.reload(qkmed)

In [ ]:
def train_qkmedians(latent_dim, train_size, read_file, seed=None, k=2, tolerance=1.e-3, save_dir=None):

    # read train data
    with h5py.File(read_file, 'r') as file:
        data = file['latent_space']
        l1 = data[:,0,:]
        l2 = data[:,1,:]

        data_train = np.vstack([l1[:train_size], l2[:train_size]])
        if seed: np.random.seed(seed) # matter for small data sizes
        np.random.shuffle(data_train)

    # train qkmedians
    centroids = qkmed.initialize_centroids(data_train, k)   # Intialize centroids

    i = 0
    while True:
        cluster_label, _ = qkmed.find_nearest_neighbour_DI(data_train,centroids)       # find nearest centroids
        print(f'Found cluster assignments for iteration: {i+1}')
        new_centroids = qkmed.find_centroids_GM(data_train, cluster_label, clusters=k) # find centroids

        loss_epoch = np.linalg.norm(centroids - new_centroids)

        if loss_epoch < tolerance:
            centroids = new_centroids
            print(f"Converged after {i+1} iterations.")
            break
        i += 1     
        centroids = new_centroids

    np.save(f'{save_dir}/cluster_label_lat{latent_dim}_{str(train_size)}.npy', cluster_label)
    np.save(f'{save_dir}/centroids_lat{latent_dim}_{str(train_size)}.npy', centroids)
    print('Centroids and labels saved!')

In [ ]:
train_qkmedians(8, 100, f'/eos/user/e/epuljak/private/epuljak/public/AE_data/latent/lat8/latentrep_QCD_sig.h5', seed=1234, save_dir='/eos/user/e/epuljak/private/epuljak/PhD/TN/QIBO/search_algorithms/notebooks')